In [2]:
import pandas as pd

# Load the data
pivot_df = pd.read_csv('/Users/balmeru/Downloads/expected_dates_1985.csv', index_col='quarter')
df2 = pd.read_csv('/Users/balmeru/Downloads/median_fyrchange_dates.csv')
df2['rdq'] = pd.to_datetime(df2['rdq'], errors='coerce')

# Fill NaN values in 'txdbq' and 'dvpq' columns
df2['txdbq'] = df2['txdbq'].fillna(0)
df2['dvpq'] = df2['dvpq'].fillna(0)

# Create 'income_good' column based on the condition
df2.loc[:, "income_good"] = False
df2.loc[~df2["ibq"].isna(), "income_good"] = True

# Create 'se1_good' column based on the condition
df2.loc[:, "se1_good"] = False
df2.loc[~df2["teqq"].isna(), "se1_good"] = True

# Create 'se2_good' column based on the condition
df2.loc[:, "se2_good"] = False
df2.loc[~df2["ceqq"].isna() & ~df2["pstkq"].isna(), "se2_good"] = True

# Create 'se3_good' column based on the condition
df2.loc[:, "se3_good"] = False
df2.loc[~df2["atq"].isna() & ~df2["ltq"].isna(), "se3_good"] = True

# Filter the DataFrame based on the given conditions
filtered_df = df2[(df2['fyr_change_dummy'] == 1) &
                  (df2['income_good'] == True) &
                  ((df2['se1_good'] == True) |
                   (df2['se2_good'] == True) |
                   (df2['se3_good'] == True))]
def update_pivot(pivot_df, filtered_df, quarters):
    for quarter in quarters:
        for tic in filtered_df['tic'].unique():
            # Find the rows for the current quarter and ticker
            condition = (filtered_df['datafqtr'] == quarter) & (filtered_df['tic'] == tic)
            relevant_rows = filtered_df[condition]

            if not relevant_rows.empty:
                # Get the expected date (assuming there should be only one per quarter-ticker pair)
                expected_date = relevant_rows['expected_date'].values[0]

                # Check if the ticker exists in the pivot table, if not, create a new column
                if tic not in pivot_df.columns:
                    pivot_df[tic] = None
                
                # Update the pivot table with the expected date
                pivot_df.loc[quarter, tic] = expected_date

    return pivot_df

# List of quarters to check
quarters = ['1985Q1', '1985Q2', '1985Q3', '1985Q4']

# Update the pivot table with the expected dates
updated_pivot_df = update_pivot(pivot_df, filtered_df, quarters)

# Save the updated pivot table to a new CSV file
updated_pivot_df.to_csv('/Users/balmeru/Downloads/updated_1985.csv')

print("Pivot table updated successfully.")


Pivot table updated successfully.
